# Questions to ask before you start:

1. Did you download the latest copy of the ADM Ops Log from Teams?
2. Do you have a file of the input poses ready?

## 0 Parameter Definitions

#### 0.0.1 Parameters you'll need to tweak

In [1]:
baseline_ADM_plateau_name = 'BA8' #Plateau name for the baseline ADM data.
update_ADM_plateau_name = 'BA9'   #Plateau name for the updated ADM data
baseline_fname = 'files/_temp input poses.xlsx' #What is the name of the input file? Don't forget to add "files/" if it's located in that subdirectory

output_filename = '_temp output poses.xlsx' #What do you want to name the output file? 
#Remember the desired format is "Poses - [name of new circuit, i.e. BA2C3] ADM update from [name of baseline circuit, i.e. BA2C1] - [Your name].xlsx"

#### 0.0.2 Parameters you probably won't need to tweak

In [2]:
pat_target = 'B'         #Which PAT target do you want to use for the update? 
z_target = 'mirror'      #Do you want range data to come from the mirror or the retro?
optimize_PAT_for_imperfect_nulling = True #Optimize the encoder positions for imperfect nulling between sMATF->PAT?

#Name of the tab in th excel file that contains the data for your baseline poses. 
#If you're using a spreadsheet that was generated using this code, you'll want to use the 'update' tab. 
#If you're using a spreadsheet that came from Manal, chances are the tab is named 'input for Tyler analysis' or something
sheetname = 'update'

### 0.1 Ingesting values from the template

In [3]:
%matplotlib notebook
import os
from adm_library import *

In [4]:
adm_log_filepath = 'files/ADM Ops Log.xlsx'

spreadsheet = pd.read_excel(baseline_fname,sheet_name=sheetname,skiprows=1,usecols='B:N')

focal_length = spreadsheet.iloc[0][0]-110 # focal length is track length -110mm, per TG
sMPA_to_WCS = spreadsheet.iloc[3,:3].astype(float)
sMPA_angle_to_WCS_deg = spreadsheet.iloc[6,:3].astype(float)
GSA_angle_WCS_deg = spreadsheet.iloc[7][0]
sMask=np.array(spreadsheet.iloc[10,1:4]).astype(float)

spreadsheet = pd.read_excel(baseline_fname,sheet_name=sheetname,skiprows=15,usecols='B:S')
pose_actual = spreadsheet.iloc[0:5,0:6]
pose_actual = pose_actual.set_index('Name').sort_index()
pose_actual.columns = ['X','Y','Z','Rx','Ry']
X_5DOF,Y_5DOF,Z_5DOF,rx_deg_5DOF,ry_deg_5DOF = [[*pose_actual['X'].values.astype(float)],[*pose_actual['Y'].values.astype(float)],[*pose_actual['Z'].values.astype(float)],
                                                [*pose_actual['Rx'].values.astype(float)],[*pose_actual['Ry'].values.astype(float)]]

#Depending on the origin of the input spreadsheet, the desired encoder values are in a different place.
if sheetname == 'Input for Tyler analysis':
    pose_encoders = spreadsheet.iloc[0:5,7:13]
elif sheetname == 'update':
    pose_encoders = spreadsheet.iloc[0:5,12:18]
    
#Rename the columns because by default, each column header got a ".1" added to the end
new_columns=[]
for column in pose_encoders.columns:
    new_columns.append(column[:-2])
pose_encoders.columns = new_columns
pose_encoders = pose_encoders.set_index('Name').sort_index()

print('Provided 5DOF position in real space: \n',pose_actual,'\n')
print('Provided 5DOF encoder values: \n',pose_encoders)


Provided 5DOF position in real space: 
                X          Y          Z        Rx         Ry
Name                                                       
PR1     1.129985    0.72747   1.232735  0.068028   0.026284
PR2  -109.321459 -46.437929  23.894719    -4.875  10.671451
PR3   111.493376 -45.937709  23.616555 -4.820656 -10.609137
PR4   103.256807  59.195421  32.320005  5.344696  -10.32082
PR5  -101.250577  58.826217  32.401763  5.306222   10.40122 

Provided 5DOF encoder values: 
                X          Y          Z        Rx         Ry
Name                                                       
PR1     1.106244   0.769616   1.212657  0.068746   0.028368
PR2  -109.227043 -46.488631  23.780078 -4.874928  10.670479
PR3   111.446144 -45.628079  23.705093 -4.819084 -10.608861
PR4   103.065882  59.420898  32.509522  5.349615 -10.318354
PR5  -101.308187  58.694581  32.398166  5.305656   10.40208


### 0.1.1 If both the pose_actual and pose_encoder arrays are provided in the spreadsheet above, convert between the two to make sure they both agree. 

In [5]:
#No point doing the following if one or the other array is completely empty
if (not any(np.isfinite(pose_encoders.values.astype(float)).ravel())) and (not any(np.isfinite(pose_actual.values.astype(float)).ravel())):

    print('(Calculated 5DOF position) - (Actual 5DOF position (from the spreadsheet)): ')
    calculated_pose_actual = pd.DataFrame(columns=pose_actual.columns)
    for index in [val for val in pose_actual.index if ('PR' in val) or ('PD' in val)]:
        calculated_pose_actual = pd.concat((calculated_pose_actual,calculate_5DOF_from_encoders(pd.DataFrame(pose_encoders.loc[index]).T)))
    print(calculated_pose_actual-pose_actual,'\n\n')
    
    print('(Calculated 5DOF encoders) - (Actual 5DOF encoders (from the spreadsheet)): ')
    calculated_pose_encoders = pd.DataFrame(columns=pose_encoders.columns)
    for index in [val for val in pose_encoders.index if ('PR' in val) or ('PD' in val)]:
        calculated_pose_encoders = pd.concat((calculated_pose_encoders,calculate_encoders_from_5DOF(pd.DataFrame(pose_actual.loc[index]).T)))
    print(calculated_pose_encoders-pose_encoders)

### 0.2 If the "pose_actual" section of the supplied spreadsheet was empty, build a new "pose_actual" array based on the input "pose_encoder" array.

In [6]:
if not any(np.isfinite(pose_actual.values.astype(float)).ravel()):
    pose_actual = convert_pose_encoders_to_pose_actual(pose_encoders)
    print('NEWLY calculated 5DOF position in real space based on recorded 5DOF encoder values: \n',pose_actual)

# 1. Updating Poses with ADM Data

## 1.1 ADM Baseline  

#### 1.1.1 Input your baseline ADM values

In [7]:
p_null_PAT_baseline_encoder = get_data_from_ADM_log(baseline_ADM_plateau_name,z_type=z_target,pat_target = pat_target,index_name='PAT'+pat_target,print_details=True,filepath=adm_log_filepath) 
p_null_PAT_baseline = calculate_5DOF_from_encoders(p_null_PAT_baseline_encoder)

if pat_target=='B':
    p_null_offset = np.array([-90.,-110.,-10.]) #PATB is not on the POA boresight. It is offset by this much
elif pat_target=='A':
    p_null_offset = np.array([90.,-110.,-10.])
p_null_PAT_baseline_encoder


X                               88.796
Y                              108.686
Z                                0.002
Rx                              0.1047
Ry                              0.0822
z_sMATF                       4024.114
z_PAT                         3474.389
z_type                          mirror
sMATF AC AZ                    -0.0042
sMATF AC EL                     0.0056
PAT AC AZ                      -0.0049
PAT AC EL                       0.0056
sMATF Pri LED X               813.1055
sMATF Pri LED Y                597.623
PAT Pri LED X                  812.201
PAT Pri LED Y                  598.029
date               2023-06-15 00:00:00
Name: PATB, dtype: object


,X,Y,Z,Rx,Ry,z_sMATF,z_PAT,z_type,sMATF AC AZ,sMATF AC EL,PAT AC AZ,PAT AC EL,sMATF Pri LED X,sMATF Pri LED Y,PAT Pri LED X,PAT Pri LED Y,date
PATB,88.796,108.686,0.002,0.1047,0.0822,4024.114,3474.389,mirror,-0.0042,0.0056,-0.0049,0.0056,813.1055,597.623,812.201,598.029,2023-06-15


### 1.1.2 Update the PAT pose to account for imperfect nulling.

In [8]:
#Apply a correction to account for imperfect nulling between PATB/sMATF and overwrite the values in the p_null_PAT_baseline_encoder DataFrame with the optimal ones.

#Save a copy of the p_null_PATB dataframe, which we'll write to an Excel file later
p_null_PAT_baseline_encoder_original = p_null_PAT_baseline_encoder.copy()
if optimize_PAT_for_imperfect_nulling == True:
    print(p_null_PAT_baseline_encoder.squeeze(),'\n')
    p_null_PAT_baseline_encoder = optimize_p_null_PAT_encoders(p_null_PAT_baseline_encoder,p_null_offset)
    p_null_PAT_baseline = calculate_5DOF_from_encoders(p_null_PAT_baseline_encoder)
    print('\n',p_null_PAT_baseline_encoder.squeeze())

X                               88.796
Y                              108.686
Z                                0.002
Rx                              0.1047
Ry                              0.0822
z_sMATF                       4024.114
z_PAT                         3474.389
z_type                          mirror
sMATF AC AZ                    -0.0042
sMATF AC EL                     0.0056
PAT AC AZ                      -0.0049
PAT AC EL                       0.0056
sMATF Pri LED X               813.1055
sMATF Pri LED Y                597.623
PAT Pri LED X                  812.201
PAT Pri LED Y                  598.029
date               2023-06-15 00:00:00
Name: PATB, dtype: object 

Starting GSARX/RY:  0.1047 0.0822
Optimal GSARX/RY:  0.1052768882 0.081803513
Starting HTSA/VTSA:  88.796 108.686
Optimal HTSA/VTSA:  88.842839565 108.675476355

 X                             88.84284
Y                           108.675476
Z                                0.002
Rx                           

#### 1.1.5 Form a dataframe with the values supplied so far

In [9]:
#Generate a dataframe where we will store some parameters for each pose
columns = ['X','Y','Z','Rx','Ry']
df = pose_actual
df['color']='crimson'

gsa_rot = R.from_euler('x',GSA_angle_WCS_deg, degrees=True)
df.loc['sMask',['X','Y','Z']] = gsa_rot.apply(sMask)
df.loc['sMask','color'] = 'g'

dz_baseline = p_null_PAT_baseline_encoder.loc['PAT'+pat_target,'z_sMATF'] - \
    p_null_PAT_baseline_encoder.loc['PAT'+pat_target,'z_PAT']
df = pd.concat([df,p_null_PAT_baseline])
print(p_null_PAT_baseline)
df.loc['PAT'+pat_target,'color']='darkblue'

df.loc['sMPA',['X','Y','Z']] = sMPA_to_WCS.values.astype(float)
df.loc['sMPA','color']='purple'

for pose in df.index:
    update_uvec(df,pose,length=focal_length,rotangle=GSA_angle_WCS_deg)
    if 'PDI' in pose:
        df.loc[pose,'color'] = 'yellow'

# create normal vector to sMPA
vec = np.array([0,0,1])
rotmat = R.from_euler('XYZ',sMPA_angle_to_WCS_deg, degrees=True)
df.loc['sMPA',['uvec_X','uvec_Y','uvec_Z']] = rotmat.apply(vec)

#ax = plot_poses(df)
#plot_sMPA(df, ax)
df

              X           Y         Z       Rx       Ry
PATB  89.168874  108.915034 -0.680054  0.10457  0.07972


,X,Y,Z,Rx,Ry,color,uvec_X,uvec_Y,uvec_Z
PR1,1.129985,0.72747,1.232735,0.068028,0.026284,crimson,0.592742,247.333881,1268.230045
PR2,-109.321459,-46.437929,23.894719,-4.875,10.671451,crimson,239.271436,349.539658,1220.718254
PR3,111.493376,-45.937709,23.616555,-4.820656,-10.609137,crimson,-237.890300,348.452843,1221.298763
PR4,103.256807,59.195421,32.320005,5.344696,-10.32082,crimson,-231.496395,127.555056,1264.800764
PR5,-101.250577,58.826217,32.401763,5.306222,10.40122,crimson,233.280011,128.371410,1264.390393
sMask,1.404588,115.316263,588.798438,NaN,NaN,g,NaN,NaN,NaN
PATB,89.168874,108.915034,-0.680054,0.10457,0.07972,darkblue,1.797822,246.524758,1268.386439
sMPA,1.722732,248.052176,1269.442466,NaN,NaN,purple,NaN,NaN,NaN


#### 1.1.6 Calculate the center of the sMATF flat in 5DOF coordinate system (currently assumes PATB is on-axis and aligned with GSA)

In [10]:
start_coords = np.array(df.loc['PAT'+pat_target,['X','Y','Z']])

rx = df.loc['PAT'+pat_target,'Rx']
ry = df.loc['PAT'+pat_target,'Ry']
print(rx,ry)
gsa_rot = R.from_euler('XY',[rx+GSA_angle_WCS_deg,ry], degrees=True)
uvec = np.array([0.,0.,dz_baseline]+p_null_offset)
rot_uvec = gsa_rot.apply(uvec) #np.dot(gsa_rot.as_matrix(),uvec)

sMATF_coords = start_coords+rot_uvec

df.loc['sMATF',['X','Y','Z']]=sMATF_coords
df.loc['sMATF',['Rx','Ry']]=df.loc['PAT'+pat_target,['Rx','Ry']]
df.loc['sMATF','color']='orange'
update_uvec(df,'sMATF',length=dz_baseline,rotangle=GSA_angle_WCS_deg)
#ax = plot_poses(df)
#plot_sMPA(df, ax)
df

0.10457037378704849 0.07971967261278287


,X,Y,Z,Rx,Ry,color,uvec_X,uvec_Y,uvec_Z
PR1,1.129985,0.72747,1.232735,0.068028,0.026284,crimson,0.592742,247.333881,1268.230045
PR2,-109.321459,-46.437929,23.894719,-4.875,10.671451,crimson,239.271436,349.539658,1220.718254
PR3,111.493376,-45.937709,23.616555,-4.820656,-10.609137,crimson,-237.890300,348.452843,1221.298763
PR4,103.256807,59.195421,32.320005,5.344696,-10.32082,crimson,-231.496395,127.555056,1264.800764
PR5,-101.250577,58.826217,32.401763,5.306222,10.40122,crimson,233.280011,128.371410,1264.390393
sMask,1.404588,115.316263,588.798438,NaN,NaN,g,NaN,NaN,NaN
PATB,89.168874,108.915034,-0.680054,0.10457,0.07972,darkblue,1.797822,246.524758,1268.386439
sMPA,1.722732,248.052176,1269.442466,NaN,NaN,purple,NaN,NaN,NaN
sMATF,-0.080084,103.933601,550.238286,0.10457,0.07972,orange,0.764869,104.881962,539.624742


## 1.2 ADM Update

Now the system has undergone some kind of change that will cause the ADM measurements to be different, such as shimming the sWFI or cooling from ambient to operational temps. We need to use the new set of ADM sMATF/PATB measurements to update the calculated poses for PRs 1-5.

Now update all poses from this new PATB pose; this is the recipe that would be followed for real data

#### 1.2.1 Input new ADM data for the sMATF and PATB and convert to 5DOF actual space

In [11]:
p_null_PAT_update_encoder = get_data_from_ADM_log(update_ADM_plateau_name,z_type=z_target,pat_target = pat_target,index_name='PAT'+pat_target+'_update',print_details=True,filepath=adm_log_filepath) 
p_null_PAT_update = calculate_5DOF_from_encoders(p_null_PAT_update_encoder)
p_null_PAT_update

X                               88.729
Y                              108.209
Z                               -0.003
Rx                              0.1078
Ry                              0.0884
z_sMATF                       4023.998
z_PAT                         3474.449
z_type                          mirror
sMATF AC AZ                    -0.0035
sMATF AC EL                      0.003
PAT AC AZ                      -0.0032
PAT AC EL                       0.0035
sMATF Pri LED X                809.127
sMATF Pri LED Y               599.6605
PAT Pri LED X                  809.766
PAT Pri LED Y                    599.7
date               2023-06-16 00:00:00
Name: PATB_update, dtype: object


,X,Y,Z,Rx,Ry
PATB_update,89.05365,108.447932,-0.682426,0.107094,0.086316


### 1.2.2 Update the PATB pose to account for imperfect nulling

In [12]:
#Apply a correction to account for imperfect nulling between PATB/sMATF and overwrite the values in the p_null_PAT_baseline_encoder DataFrame with the optimal ones.

#Save a copy of the p_null_PATB dataframe, which we'll write to an Excel file later
p_null_PAT_update_encoder_original = p_null_PAT_update_encoder.copy()

if optimize_PAT_for_imperfect_nulling == True:
    p_null_PAT_update_encoder = optimize_p_null_PAT_encoders(p_null_PAT_update_encoder,p_null_offset)
    p_null_PAT_update = calculate_5DOF_from_encoders(p_null_PAT_update_encoder)

Starting GSARX/RY:  0.1078 0.0884
Optimal GSARX/RY:  0.1072695587 0.08815786
Starting HTSA/VTSA:  88.729 108.209
Optimal HTSA/VTSA:  88.705545765 108.22903732


#### 1.2.2 Use the newly-provided data to calculate an updated position for PATB and sMATF

In [13]:
dz_update = p_null_PAT_update_encoder.loc['PAT'+pat_target+'_update','z_sMATF'] - \
    p_null_PAT_update_encoder.loc['PAT'+pat_target+'_update','z_PAT'] + \
    (p_null_PAT_update_encoder.loc['PAT'+pat_target+'_update',['Y']]-p_null_PAT_baseline_encoder.loc['PAT'+pat_target,['Y']]).values[0]*np.sin(np.deg2rad(GSA_angle_WCS_deg+p_null_PAT_update_encoder.loc['PAT'+pat_target+'_update',['Rx']].astype(float).values[0]))
df = pd.concat([df,p_null_PAT_update])
df.loc['PAT'+pat_target+'_update','color'] = 'purple'
update_uvec(df,'PAT'+pat_target+'_update',length=focal_length,rotangle=GSA_angle_WCS_deg)

start_coords = np.array(df.loc['PAT'+pat_target+'_update',['X','Y','Z']])

rx = df.loc['PAT'+pat_target+'_update','Rx']
ry = df.loc['PAT'+pat_target+'_update','Ry']
gsa_rot = R.from_euler('XY',[rx+GSA_angle_WCS_deg,ry], degrees=True)
uvec = [0.,0.,dz_update]+p_null_offset # in GSA CS
rot_uvec = gsa_rot.apply(uvec) #np.dot(gsa_rot.as_matrix(),uvec)

sMATF_update_coords = start_coords+rot_uvec

df.loc['sMATF_update',['X','Y','Z']]=sMATF_update_coords
df.loc['sMATF_update',['Rx','Ry']]=df.loc['PAT'+pat_target+'_update',['Rx','Ry']]
df.loc['sMATF_update','color']='brown'
update_uvec(df,'sMATF_update',length=dz_update,rotangle=GSA_angle_WCS_deg)
#ax = plot_poses(df)
#plot_sMPA(df, ax)
print('Baseline poses')
df


Baseline poses


,X,Y,Z,Rx,Ry,color,uvec_X,uvec_Y,uvec_Z
PR1,1.129985,0.72747,1.232735,0.068028,0.026284,crimson,0.592742,247.333881,1268.230045
PR2,-109.321459,-46.437929,23.894719,-4.875,10.671451,crimson,239.271436,349.539658,1220.718254
PR3,111.493376,-45.937709,23.616555,-4.820656,-10.609137,crimson,-237.890300,348.452843,1221.298763
PR4,103.256807,59.195421,32.320005,5.344696,-10.32082,crimson,-231.496395,127.555056,1264.800764
PR5,-101.250577,58.826217,32.401763,5.306222,10.40122,crimson,233.280011,128.371410,1264.390393
sMask,1.404588,115.316263,588.798438,NaN,NaN,g,NaN,NaN,NaN
PATB,89.168874,108.915034,-0.680054,0.10457,0.07972,darkblue,1.797822,246.524758,1268.386439
sMPA,1.722732,248.052176,1269.442466,NaN,NaN,purple,NaN,NaN,NaN
sMATF,-0.080084,103.933601,550.238286,0.10457,0.07972,orange,0.764869,104.881962,539.624742
PATB_update,89.030176,108.468036,-0.682416,0.106564,0.086074,purple,1.941127,246.480587,1268.394812


In [14]:
#How did the distance between PATB and sMATF change from pre- to post-environment shift?
print('PAT'+pat_target+'->sMATF distance changed by: ',dz_update-dz_baseline, ' mm')

PATB->sMATF distance changed by:  -0.08847588081619051  mm


## 1.3 Calculate sMATF and PAT deltas

In [15]:
sMATF_delta = df.loc['sMATF_update',['X','Y','Z']] - df.loc['sMATF',['X','Y','Z']]
#sMATF_delta.loc['Z'] += 
print('sMATF (X,Y,Z) deltas: \n',sMATF_delta,'\n')

patb_delta = df.loc['PAT'+pat_target+'_update',['X','Y','Z','Rx','Ry']] - df.loc['PAT'+pat_target,['X','Y','Z','Rx','Ry']]
#patb_drxry = df.loc['PATB_update',['Rx','Ry']] - df.loc['PATB',['Rx','Ry']]
print('PAT'+pat_target+' (X,Y,Z,Rx,Ry) deltas: \n',patb_delta,'\n')

sMATF (X,Y,Z) deltas: 
 X   -0.078958
Y   -0.481157
Z   -0.079674
dtype: object 

PATB (X,Y,Z,Rx,Ry) deltas: 
 X    -0.138698
Y    -0.446999
Z    -0.002362
Rx    0.001993
Ry    0.006354
dtype: object 



## 1.4 Transform old poses to the new sMATF reference frame

### 1.4.1 Apply a translation to all poses

In [16]:
df_update = df.copy()
T = df.loc['sMATF_update',['X','Y','Z']]
df_update[['X','Y','Z']] -= T
print('Translation required to put sMATF_update at the origin: \n',T)

Translation required to put sMATF_update at the origin: 
 X     -0.159041
Y    103.452445
Z    550.158612
Name: sMATF_update, dtype: object


### 1.4.2 Now perform 3 rotations to all the poses:
1. To align to the sMATF frame
2. To rotate everything by delta(Rx,Ry)
3. To undo #1

In [17]:
update_rot1 = R.from_euler('XY',[df.loc['PAT'+pat_target,'Rx']+GSA_angle_WCS_deg,df.loc['PAT'+pat_target,'Ry']], degrees=True)
update_rot2 = R.from_euler('XY',[patb_delta.loc['Rx'],patb_delta.loc['Ry']], degrees=True)
update_rot_combined = update_rot1 * update_rot2 * update_rot1.inv()
                                
for pose in df_update.index:
                                
    df_update.loc[pose,['X','Y','Z']]=update_rot_combined.apply(df_update.loc[pose,['X','Y','Z']].astype(float)) #np.dot(update_rot.as_matrix(),df_update.loc[pose,['X','Y','Z']])
    df_update.loc[pose,['uvec_X','uvec_Y','uvec_Z']] = update_rot_combined.apply(df_update.loc[pose,['uvec_X','uvec_Y','uvec_Z']].astype(float)) 
    if pose not in ['sMask','sMPA']:
        update_RxRy(df_update,pose,GSA_angle_WCS_deg)


### 1.4.3 Translate back to the 5DOF frame by undoing the translation from Cell 1.4.1, followed by adding the sMATF delta(X,Y,Z) offset from Cell 1.3

In [18]:
df_update[['X','Y','Z']] += T
df_update.loc[df_update.index[:12],['X','Y','Z']] += update_rot_combined.apply(sMATF_delta.astype(float)) #np.dot(update_rot.as_matrix(),delta)
#ax = plot_poses(df_update)
#plot_sMPA(df_update, ax)
print('Updated poses')
df_update

Updated poses


,X,Y,Z,Rx,Ry,color,uvec_X,uvec_Y,uvec_Z
PR1,0.989074,0.26539,1.149343,0.070021,0.032638,crimson,0.736048,247.289741,1268.238577
PR2,-109.460903,-46.898455,23.821709,-4.87311,10.677782,crimson,239.411735,349.492110,1220.704360
PR3,111.353911,-46.402907,23.519524,-4.81856,-10.602805,crimson,-237.749958,348.415393,1221.336775
PR4,103.120519,58.730094,32.227528,5.346584,-10.314492,crimson,-231.356002,127.515957,1264.830395
PR5,-101.386863,58.365223,32.331537,5.308322,10.407549,crimson,233.420373,128.322470,1264.369456
sMask,1.33007,114.833733,588.718998,NaN,NaN,g,NaN,NaN,NaN
PATB,89.030048,108.451154,-0.769266,0.106564,0.086074,darkblue,1.941127,246.480587,1268.394812
sMPA,1.725124,247.545957,1269.367605,NaN,NaN,purple,NaN,NaN,NaN
sMATF,-0.159041,103.452445,550.158612,0.106564,0.086074,orange,0.825837,104.863170,539.628304
PATB_update,88.891341,108.004159,-0.771629,0.108557,0.092429,purple,2.084432,246.436413,1268.403168


# 1.4 >>>>Results<<<<

#### 1.4.1 Converting poses from real space to encoder space

In [19]:
df_update_encoder_space = convert_df_to_encoder_space(df_update)
print('Poses calculated using PAT'+pat_target+' and the PAT '+z_target)
print('Updated Poses (in encoder space): \n',df_update_encoder_space,'\n\n')

print('Baseline Poses (in encoder space), just for easy reference: \n',pose_encoders)

Poses calculated using PATB and the PAT mirror
Updated Poses (in encoder space): 
               X          Y          Z        Rx         Ry
PR1    0.966019   0.306395   1.129517  0.070739   0.034722
PR2 -109.365072 -46.955212  23.710647 -4.873037  10.676808
PR3  111.308016 -46.097285  23.609685 -4.816990 -10.602527
PR4  102.930345  58.957892  32.414454  5.351499 -10.312024
PR5 -101.443629  58.233845  32.327236  5.307756  10.408406 


Baseline Poses (in encoder space), just for easy reference: 
                X          Y          Z        Rx         Ry
Name                                                       
PR1     1.106244   0.769616   1.212657  0.068746   0.028368
PR2  -109.227043 -46.488631  23.780078 -4.874928  10.670479
PR3   111.446144 -45.628079  23.705093 -4.819084 -10.608861
PR4   103.065882  59.420898  32.509522  5.349615 -10.318354
PR5  -101.308187  58.694581  32.398166  5.305656   10.40208


#### (OPTIONAL) 1.4.2 Display some other tables that may be helpful for diagnostics

In [20]:
print_extra_diagnostics = False
poses_to_display = [val for val in df.index if 'PDI' in val or 'PR' in val]+['sMask', 'sMPA']

if print_extra_diagnostics == True:
    print('Baseline Poses (in real space): \n',df.loc[poses_to_display,['X','Y','Z','Rx','Ry']],'\n\n')
    print('Updated Poses (in real space): \n',df_update.loc[poses_to_display,['X','Y','Z','Rx','Ry']], '\n\n')
    print('Deltas between Baseline/Updated poses (in real space): \n',-df.loc[poses_to_display,['X','Y','Z','Rx','Ry']] + df_update.loc[poses_to_display,['X','Y','Z','Rx','Ry']], '\n\n')
    print('Change in calculated endpoint positions (5DOF frame): \n',compute_endpoint_errors(df,df_update),'\n\n')
    print('Change in calculated endpoint positions (sMPA frame): \n',compute_endpoint_errors_sMPA_frame(df,df_update),'\n\n')
    print('Verify that all poses intersect at the pupil in the same way before and after the update:')
    print(check_pupil_crossing([df,df_update]),'\n\n')
    print('Distance from pose focus to sMPA: \n\n',compute_distance_to_sMPA(df),'\n\n')  
    

# 2 Write the new poses to an Excel file in the current directory

In [21]:
write_to_excel = True

if write_to_excel == True:
    write_new_poses_to_Excel(output_filename,'',update_type='ADM',baseline_filepath=baseline_fname,
                             columns=columns,GSA_angle_WCS_deg=GSA_angle_WCS_deg,
                             baseline_ADM_plateau_name=baseline_ADM_plateau_name,update_ADM_plateau_name=update_ADM_plateau_name,
                             df=df,df_encoders=pose_encoders,df_update=df_update,df_update_encoders=df_update_encoder_space,
                            focal_length=focal_length,p_null_PAT_baseline_encoder_original=p_null_PAT_baseline_encoder_original,
                            p_null_PAT_update_encoder_original=p_null_PAT_update_encoder_original)

**Writing to Excel complete.**
**Filename:  _temp output poses.xlsx
